# NLP - Modelos de linguagem

### Bibliotecas básicas e outros imports

In [ ]:
!pip install nltk==3.5

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10, 6)
mpl.rcParams['font.size'] = 14

In [ ]:
import nltk
nltk.download('all')

from nltk import tokenize
from nltk.util import bigrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.corpus import stopwords
STOP_WORDS = stopwords.words('portuguese')

### Carregando os dados

In [6]:
dados_portugues = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Alura/stackoverflow_portugues.csv')
dados_portugues.head(2)

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953


In [7]:
print(dados_portugues['Questão'][5])

<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>

<pre><code>//----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
</code></pre>

<p>Logo, digamos que o usuário usar a sentença: <code>1); DROP TABLE Produtos;</code> para ao campo <code>valor</code> o comando ficaria: </p>

<pre><code>insert into Produtos (coluna) values(1); DROP TABLE Produtos;
</code></pre>

<p>Ele vai inserir um novo registro cujo o campo <code>coluna</code> será <code>1</code> e logo em seguida ele vai deletar a tabela Produtos.</p>

<p>Como posso melhorar meu código para prevenir essa situação?</p>



In [8]:
dados_ingles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Alura/stackoverflow_ingles.csv')
dados_ingles.head(2)

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137


In [9]:
print(dados_ingles['Questão'][0])

<p>Here is a piece of C++ code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>

<pre class="lang-cpp prettyprint-override"><code>#include &lt;algorithm&gt;
#include &lt;ctime&gt;
#include &lt;iostream&gt;

int main()
{
    // Generate data
    const unsigned arraySize = 32768;
    int data[arraySize];

    for (unsigned c = 0; c &lt; arraySize; ++c)
        data[c] = std::rand() % 256;

    // !!! With this, the next loop runs faster
    std::sort(data, data + arraySize);

    // Test
    clock_t start = clock();
    long long sum = 0;

    for (unsigned i = 0; i &lt; 100000; ++i)
    {
        // Primary loop
        for (unsigned c = 0; c &lt; arraySize; ++c)
        {
            if (data[c] &gt;= 128)
                sum += data[c];
        }
    }

    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / CLOCKS_PER_SEC;

    std::cout &lt;&lt; elapsedTime &lt;&lt; std::endl;
    std::cout &lt

In [10]:
dados_espanhol = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Alura/stackoverflow_espanhol.csv')
dados_espanhol.head(2)

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860


In [11]:
print(dados_espanhol['Questão'][0])

<p>Las sentencias dinámicas son sentencias SQL que se crean como cadenas de texto (strings) y en las que se insertan/concatenan valores obtenidos de alguna fuente (normalmente proveniente del usuario), lo que puede hacer que sean vulnerables a inyección SQL si no se sanean las entradas, como por ejemplo:</p>

<pre><code>$id_usuario = $_POST["id"];

mysql_query("SELECT * FROM usuarios WHERE id = $id_usuario");
</code></pre>

<p>Eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación (web o no) porque si el usuario introdujese un valor como <code>1; DROP TABLE usuarios;--</code> nos encontraríamos con que la sentencia ejecutada sería:</p>

<pre><code>SELECT * FROM usuarios WHERE id = 1; DROP TABLE usuarios;--
</code></pre>

<p>Y se eliminaría la tabla Usuarios con todos los datos contenidos en ella. </p>

<p><strong>¿Cómo puedo evitar que la inyección SQL ocurra en PHP?</strong></p>



### Removendo Tag HTML

In [12]:
def remover(textos, regex):
  if type(textos) == str:
    return regex.sub(' ', textos  )
  else:
    return [regex.sub(' ', texto) for texto in textos]

remove_tag_regex = re.compile(r'<.*?>')
remover(dados_portugues['Questão'][5], remove_tag_regex)

" Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo: \n\n  //----CONSULTA SQL----//\n$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');\n  \n\n Logo, digamos que o usuário usar a sentença:  1); DROP TABLE Produtos;  para ao campo  valor  o comando ficaria:  \n\n  insert into Produtos (coluna) values(1); DROP TABLE Produtos;\n  \n\n Ele vai inserir um novo registro cujo o campo  coluna  será  1  e logo em seguida ele vai deletar a tabela Produtos. \n\n Como posso melhorar meu código para prevenir essa situação? \n"

### Removendo trechos de código

In [13]:
def substituir_codigo(textos, regex):
  if type(textos) == str:
    return regex.sub('CODE', textos  )
  else:
    return [regex.sub('CODE', texto) for texto in textos]
  
remove_code_regex = re.compile(r'<code>(.|(\n))*?</code>')
substituir_codigo(dados_portugues['Questão'][5], remove_code_regex)

'<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>\n\n<pre>CODE</pre>\n\n<p>Logo, digamos que o usuário usar a sentença: CODE para ao campo CODE o comando ficaria: </p>\n\n<pre>CODE</pre>\n\n<p>Ele vai inserir um novo registro cujo o campo CODE será CODE e logo em seguida ele vai deletar a tabela Produtos.</p>\n\n<p>Como posso melhorar meu código para prevenir essa situação?</p>\n'

### Removendo as pontuações

In [14]:
remove_pontuacao_regex = re.compile(r'[^\w\s]')
remover(dados_portugues['Questão'][5], remove_pontuacao_regex)

' p Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam  Através dessa página é possível fazer algumas consultas  inserções  alterações e remoções de dados de uma tabela em um banco de dados MySQL  porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL  por exemplo   p \n\n pre  code       CONSULTA SQL      \n busca   mysql_query   insert into Produtos  coluna  values      valor        \n  code   pre \n\n p Logo  digamos que o usuário usar a sentença   code 1   DROP TABLE Produtos   code  para ao campo  code valor  code  o comando ficaria    p \n\n pre  code insert into Produtos  coluna  values 1   DROP TABLE Produtos \n  code   pre \n\n p Ele vai inserir um novo registro cujo o campo  code coluna  code  será  code 1  code  e logo em seguida ele vai deletar a tabela Produtos   p \n\n p Como posso melhorar meu código para prevenir essa situação   p \n'

### Removendo os dígitos

In [15]:
remove_digitos_regex = re.compile(r'\d+')
remover(dados_portugues['Questão'][5], remove_digitos_regex)

"<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>\n\n<pre><code>//----CONSULTA SQL----//\n$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');\n</code></pre>\n\n<p>Logo, digamos que o usuário usar a sentença: <code> ); DROP TABLE Produtos;</code> para ao campo <code>valor</code> o comando ficaria: </p>\n\n<pre><code>insert into Produtos (coluna) values( ); DROP TABLE Produtos;\n</code></pre>\n\n<p>Ele vai inserir um novo registro cujo o campo <code>coluna</code> será <code> </code> e logo em seguida ele vai deletar a tabela Produtos.</p>\n\n<p>Como posso melhorar meu código para prevenir essa situação?</p>\n"

### Removendo espaços duplicados

In [16]:
remove_espaco_regex = re.compile(r' +')
remove_quebra_de_linha_regex = re.compile(r'(\n)')

def substituir_espaco(textos, regex):
    if type(textos) == str:
      return regex.sub(' ', textos  )
    else:
      return [regex.sub(' ', texto) for texto in textos]

substituir_espaco(dados_portugues['Questão'][21], remove_espaco_regex)

'<p>Para facilitar a busca de informações comumente grafadas erradas, usamos <a href="http://pt.wikipedia.org/wiki/String_Fon%C3%A9tica" rel="noreferrer">algoritmos fonéticos</a>. Um recurso que é extremamente útil mas que costuma ser negligenciado, especialmente fora da língua inglesa onde existem alguns algoritmos conhecidos como os citados na <a href="http://en.wikipedia.org/wiki/Phonetic_algorithm" rel="noreferrer">Wikipedia</a><sup>(en)</sup>, particularmente o <strong>Soundex</strong> disponível em diversos DBs e linguagens.</p>\n\n<p>Temos particularidades na nossa língua que inviabilizam o uso de algoritmos de outras línguas. Na verdade é até mais regionalizado. Um <strong>algoritmo que funciona para o Brasil</strong>, não funciona para Portugal e talvez para outros países lusófonos. Tenho até dúvidas se precisaria de especializações para o <a href="http://pt.wikipedia.org/wiki/Dialeto_da_costa_norte" rel="noreferrer">cearense</a>, <a href="http://pt.wikipedia.org/wiki/Dialeto_

### Limpando os dados

In [17]:
def cleaning(textos):
  textos = textos.lower()
  textos = substituir_codigo(textos, remove_code_regex)
  textos = remover(textos, remove_tag_regex)
  textos = remover(textos, remove_pontuacao_regex)
  textos = remover(textos, remove_digitos_regex)
  textos = substituir_espaco(textos, remove_quebra_de_linha_regex)
  textos = substituir_espaco(textos, remove_espaco_regex)
  return textos

In [18]:
dados_portugues['cleaning'] = dados_portugues['Questão'].apply(cleaning)
print(dados_portugues['cleaning'][5])

 desenvolvi uma página em php para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam através dessa página é possível fazer algumas consultas inserções alterações e remoções de dados de uma tabela em um banco de dados mysql porém eu acredito que meu código em php não está protegido contra injeção de código sql por exemplo CODE logo digamos que o usuário usar a sentença CODE para ao campo CODE o comando ficaria CODE ele vai inserir um novo registro cujo o campo CODE será CODE e logo em seguida ele vai deletar a tabela produtos como posso melhorar meu código para prevenir essa situação 


In [19]:
dados_ingles['cleaning'] = dados_ingles['Questão'].apply(cleaning)
print(dados_ingles['cleaning'][15])

 i have an array of numbers and i m using the CODE method to add elements to it is there a simple way to remove a specific element from an array the equivalent of something like CODE i have to use core javascript no frameworks are allowed 


In [20]:
dados_espanhol['cleaning'] = dados_espanhol['Questão'].apply(cleaning)
print(dados_espanhol['cleaning'][0])

 las sentencias dinámicas son sentencias sql que se crean como cadenas de texto strings y en las que se insertan concatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección sql si no se sanean las entradas como por ejemplo CODE eso es un ejemplo de una vulnerabilidad grave en la seguridad de una aplicación web o no porque si el usuario introdujese un valor como CODE nos encontraríamos con que la sentencia ejecutada sería CODE y se eliminaría la tabla usuarios con todos los datos contenidos en ella cómo puedo evitar que la inyección sql ocurra en php 


### Separação dos dados

In [29]:
from sklearn.model_selection import train_test_split

dados_portugues['idioma'] = 'port'
dados_espanhol['idioma'] = 'esp'
dados_ingles['idioma'] = 'ing'

port_treino, port_teste = train_test_split(dados_portugues['cleaning'], test_size=0.2, random_state=123)
esp_treino, esp_teste = train_test_split(dados_espanhol['cleaning'], test_size=0.2, random_state=123)
ing_treino, ing_teste = train_test_split(dados_ingles['cleaning'], test_size=0.2, random_state=123)

### Pipeline de treinamento

In [35]:
from nltk.lm.preprocessing import padded_everygram_pipeline

todas_questoes_port = ' '.join(port_treino)
todas_as_palavras_port = tokenize.WhitespaceTokenizer().tokenize(todas_questoes_port)

port_treino_bigram, vocab_port = padded_everygram_pipeline(2, todas_as_palavras_port)

### Modelo de linguagem MLE: máxima verossimilhança

In [39]:
from nltk.lm import MLE
modelo_port = MLE(2)
modelo_port.fit(port_treino_bigram, vocab_port)

In [43]:
from nltk.lm import NgramCounter
print(modelo_port.generate(num_words=6))
print(modelo_port.counts[['m']].items())

['r', '</s>', 'r', 'a', 'm', 'a']
dict_items([('</s>', 2756), ('a', 2051), ('b', 143), ('p', 719), ('o', 854), ('e', 1503), ('u', 227), ('v', 13), ('í', 13), ('i', 290), ('ó', 37), ('â', 11), ('l', 22), ('é', 76), ('ú', 5), ('ã', 4), ('á', 32), ('s', 8), ('g', 4), ('_', 1), ('m', 23), ('y', 9), ('d', 2)])
